# Price Data Extraction for Post-Index Rebalancing Arbitrage Strategy

Data source: Refinitiv Datastream via WRDS

This notebook executes codes to extract relevant data, based on the historical records of FTSE100 and FTSE250 rebalancing, for the past 10 years (2013Q1 - 2023Q3).
In this exercise, the following stocks are excluded:
- Stocks that are suspended from trading within the analysis period (+/- 20 days from rebalancing date)
- Stocks which rebalancing dates fall within the announcement date and the ex date of a corporate action
- Stocks that we are unable to obtain a reliable historical data on
- All Q3 2023 rebalancing; at time of study we are unable to obtain 20 days after the rebalancing date

In [1]:
# Import WRDS library
import wrds
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os

## Data Extraction

In [2]:
def read_sql_script(fname):
    fd = open(fname, 'r')
    sqlFile = fd.read()
    fd.close()

    return sqlFile


# Get current path
current_dir = os.getcwd()


# Define sql file names
# these will be used as a global variable
query_historical_prices = read_sql_script('../SQL/get_historical_prices.sql')
query_shares_outstanding = read_sql_script('../SQL/get_shares_outstanding.sql')

# Establish live connection; requires user login (passwords will be masked)
db = wrds.Connection() # this will be used as a global variable


def get_historical_prices(isin, start_date, end_date):
    
    print(f'Extracting historical prices for {isin}...')

    df =\
    (
        db
        .raw_sql(
            query_historical_prices.format(isin, start_date, end_date), 
            date_cols = ['trade_date']
            )
    )

    if df.empty:
        print('Dataframe is empty. No results was returned!')
    
    print('--------------------------------------------------')

    return df


WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
ftse_rebal = pd.read_csv('../constituent_history/ftse_10y_rebal_records.csv')
ftse_rebal.head()

,Post Date,Name,ISIN,FTSE100,FTSE250
0,18/9/2023,888 Holdings,GI000A0F6407,NaN,1.0
1,18/9/2023,Abrdn,GB00BF8Q6K64,-1.0,1.0
2,18/9/2023,Breedon Group,GB00BM8NFJ84,NaN,1.0
3,18/9/2023,CAB Payments Holdings,GB00BMCYKB41,NaN,1.0
4,18/9/2023,Capita,GB00B23K0M20,NaN,-1.0


In [4]:
look_back = 40
look_forward = 40

ftse_rebal["Post Date"] =\
    pd.to_datetime(ftse_rebal["Post Date"], 
                   format = '%d/%m/%Y')

ftse_rebal["start_date"] =\
    (
        ftse_rebal["Post Date"] - timedelta(days = look_back)
    ).dt.strftime('%d/%m/%Y')

ftse_rebal["end_date"] =\
    (
        ftse_rebal["Post Date"] + timedelta(days = look_back)
    ).dt.strftime('%d/%m/%Y')

In [5]:
target_isins = ftse_rebal["ISIN"]
start_dates = ftse_rebal["start_date"]
end_dates = ftse_rebal["end_date"]   


historical_prices =\
    (
        pd.
        concat(
            map(
                get_historical_prices,
                target_isins,
                start_dates,
                end_dates
            )
        )
    )

Extracting historical prices for GI000A0F6407...
--------------------------------------------------
Extracting historical prices for GB00BF8Q6K64...
--------------------------------------------------
Extracting historical prices for GB00BM8NFJ84...
--------------------------------------------------
Extracting historical prices for GB00BMCYKB41...
--------------------------------------------------
Extracting historical prices for GB00B23K0M20...
--------------------------------------------------
Extracting historical prices for GB00BG5KQW09...
--------------------------------------------------
Extracting historical prices for GB00B14SKR37...
--------------------------------------------------
Extracting historical prices for GB0009633180...
--------------------------------------------------
Extracting historical prices for GB0001826634...
--------------------------------------------------
Extracting historical prices for GG00BMD8MJ76...
--------------------------------------------------


In [6]:
historical_prices.head()

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume
0,2023-08-09,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.066,1.136,1.063000,1.117,358252.0
1,2023-08-10,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.140,1.140,1.091000,1.120,284950.0
2,2023-08-11,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.331,1.098000,1.150,1003896.0
3,2023-08-14,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.147,1.160,1.088108,1.096,1088784.0
4,2023-08-15,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.126,1.001000,1.114,1127118.0


## Data Cleaning

In [7]:
# Here, we only filter for stocks that are listed on LSEG
# There are stocks that somehow the datastream returns the primary stock listed on other exchanges
# Those stocks should not be part of the analysis

lse_historical_prices = historical_prices.loc[historical_prices.primary_exchange == 'LON', :].copy()
lse_historical_prices.close.isna().sum()

0

In [8]:
rebal_round = {
    1 : 'Q4',
    2 : 'Q1',
    3 : 'Q1',
    4 : 'Q1',
    5 : 'Q2',
    6 : 'Q2',
    7 : 'Q2',
    8 : 'Q3',
    9 : 'Q3',
    10 : 'Q3',
    11 : 'Q4',
    12 : 'Q4',
}

In [9]:
lse_historical_prices['year'] = lse_historical_prices['trade_date'].dt.year
lse_historical_prices['month'] = lse_historical_prices['trade_date'].dt.month
lse_historical_prices['rebal'] =\
(
    (lse_historical_prices['year'] 
     - 1*(lse_historical_prices['month'] == 1)).astype(str)
    + lse_historical_prices['month'].map(rebal_round)
)

lse_historical_prices.head()

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume,year,month,rebal
0,2023-08-09,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.066,1.136,1.063000,1.117,358252.0,2023,8,2023Q3
1,2023-08-10,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.140,1.140,1.091000,1.120,284950.0,2023,8,2023Q3
2,2023-08-11,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.331,1.098000,1.150,1003896.0,2023,8,2023Q3
3,2023-08-14,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.147,1.160,1.088108,1.096,1088784.0,2023,8,2023Q3
4,2023-08-15,18982.0,888 HOLDINGS,LON,26862.0,GI000A0F6407,GBP,1.100,1.126,1.001000,1.114,1127118.0,2023,8,2023Q3


In [10]:
# Remove Q3 2023 Rebal due to incomplete data
lse_historical_prices =\
    lse_historical_prices[lse_historical_prices['rebal'] != '2023Q3']

lse_historical_prices.head()

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume,year,month,rebal
0,2023-05-10,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,6.310000,6.390098,4.873999,4.873999,3884045.0,2023,5,2023Q2
1,2023-05-11,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,4.917998,5.159998,4.449998,5.000000,6081492.0,2023,5,2023Q2
2,2023-05-12,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,5.050000,5.482000,4.964507,5.050000,3685914.0,2023,5,2023Q2
3,2023-05-15,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,4.649998,4.800000,3.805950,4.005000,5816469.0,2023,5,2023Q2
4,2023-05-16,4801.0,ASOS,LON,9204.0,GB0030927254,GBP,3.960000,4.273999,3.932000,3.988999,1900093.0,2023,5,2023Q2


In [11]:
lse_historical_prices =\
    lse_historical_prices\
    .sort_values(by = ['security_name', 'trade_date'])\
    .reset_index(drop = True)

In [12]:
lse_historical_prices

,trade_date,security_code,security_name,primary_exchange,refinitiv_code,isin_code,currency,open,high,low,close,volume,year,month,rebal
0,2014-05-14,56160.0,3I GROUP,LON,74231.0,GB00B1YW4409,GBP,3.799998,4.049998,3.799998,4.002000,2346384.0,2014,5,2014Q2
1,2014-05-15,56160.0,3I GROUP,LON,74231.0,GB00B1YW4409,GBP,4.038999,4.053999,3.862000,3.875999,1717678.0,2014,5,2014Q2
2,2014-05-16,56160.0,3I GROUP,LON,74231.0,GB00B1YW4409,GBP,3.879998,3.900999,3.751499,3.792998,3144640.0,2014,5,2014Q2
3,2014-05-19,56160.0,3I GROUP,LON,74231.0,GB00B1YW4409,GBP,3.798999,3.888399,3.755999,3.810999,3537284.0,2014,5,2014Q2
4,2014-05-20,56160.0,3I GROUP,LON,74231.0,GB00B1YW4409,GBP,3.817998,3.916199,3.814998,3.902998,2375837.0,2014,5,2014Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31355,2014-10-27,206528.0,ZPG,LON,276558.0,GB00BMHTHT14,GBP,2.070000,2.118999,2.023999,2.058000,81461.0,2014,10,2014Q3
31356,2014-10-28,206528.0,ZPG,LON,276558.0,GB00BMHTHT14,GBP,2.056999,2.112000,2.042999,2.049999,270533.0,2014,10,2014Q3
31357,2014-10-29,206528.0,ZPG,LON,276558.0,GB00BMHTHT14,GBP,2.042000,2.101000,2.009000,2.028999,380191.0,2014,10,2014Q3
31358,2014-10-30,206528.0,ZPG,LON,276558.0,GB00BMHTHT14,GBP,2.044000,2.068000,2.028999,2.054999,167233.0,2014,10,2014Q3


## Get prices on required days

The required days are
- ex-ante: 20 days prior, 5 days prior, 3 days prior, 1 day prior
- ex-post: 3 days after, 5 days after, 10 days after, 20 days after

In [13]:
target_isins
rebal_dates = ftse_rebal["Post Date"].dt.strftime('%d/%m/%Y')

target_rebal_prices = []

for isin, rebal_date in zip(target_isins, rebal_dates):
    # Remove stocks that are suspended from trading during the analysis period
    if (isin, rebal_date) in [('GB00BJP5HK17', '19/12/2022'), 
                              ('GB00B1VNST91', '18/06/2018'),
                              ('GB0007892358', '19/06/2017')]:
        continue
    if lse_historical_prices[(lse_historical_prices.isin_code == isin) 
                             & (lse_historical_prices.trade_date == rebal_date)].empty:
        print(f'ISIN {isin} for {rebal_date} is excluded from studies!')
    
    else:
        sub_df = lse_historical_prices[lse_historical_prices.isin_code == isin]
        rebal_idx =\
        (
            sub_df
            .index[sub_df.trade_date == rebal_date]
            [0]
        )
        
        for delta in [-20, -5, -3, -1, 3, 5, 10, 20]:
            # Ensure that the prices for the days required exist
            assert sub_df['rebal'].loc[rebal_idx] == sub_df['rebal'].loc[rebal_idx + delta],\
            f'ISIN {isin} faced insufficient data pre-rebal on {rebal_date} for delta {delta} days'
        
        pre_20_pd = sub_df.close.loc[rebal_idx - 20]
        pre_5_pd = sub_df.close.loc[rebal_idx - 5]
        pre_3_pd = sub_df.close.loc[rebal_idx - 3]
        pre_1_pd = sub_df.close.loc[rebal_idx - 1]
        post_3_pd = sub_df.close.loc[rebal_idx + 3]
        post_5_pd = sub_df.close.loc[rebal_idx + 5]
        post_10_pd = sub_df.close.loc[rebal_idx + 10]
        post_20_pd = sub_df.close.loc[rebal_idx + 20]
        
        
        target_rebal_prices.append({
            'Name' : sub_df.security_name.values[0],
            'ISIN' : isin,
            'post_date' : rebal_date,
            'pre_twenty_pd' : pre_20_pd,
            'pre_five_pd' : pre_5_pd,
            'pre_three_pd' : pre_3_pd,
            'pre_one_pd' : pre_1_pd,
            'post_three_pd' : post_3_pd,
            'post_five_pd' : post_5_pd,
            'post_ten_pd' : post_10_pd,
            'post_twenty_pd' : post_20_pd,
        })
    

ISIN GI000A0F6407 for 18/09/2023 is excluded from studies!
ISIN GB00BF8Q6K64 for 18/09/2023 is excluded from studies!
ISIN GB00BM8NFJ84 for 18/09/2023 is excluded from studies!
ISIN GB00BMCYKB41 for 18/09/2023 is excluded from studies!
ISIN GB00B23K0M20 for 18/09/2023 is excluded from studies!
ISIN GB00BG5KQW09 for 18/09/2023 is excluded from studies!
ISIN GB00B14SKR37 for 18/09/2023 is excluded from studies!
ISIN GB0009633180 for 18/09/2023 is excluded from studies!
ISIN GB0001826634 for 18/09/2023 is excluded from studies!
ISIN GG00BMD8MJ76 for 18/09/2023 is excluded from studies!
ISIN GB00B0LCW083 for 18/09/2023 is excluded from studies!
ISIN BMG4593F1389 for 18/09/2023 is excluded from studies!
ISIN GB00BZ4BQC70 for 18/09/2023 is excluded from studies!
ISIN GB0031274896 for 18/09/2023 is excluded from studies!
ISIN GB00BY7QYJ50 for 18/09/2023 is excluded from studies!
ISIN GB00BMT9K014 for 18/09/2023 is excluded from studies!
ISIN GB0006825383 for 18/09/2023 is excluded from studie

In [14]:
ftse_tidy_data =\
    pd\
        .DataFrame(target_rebal_prices)

ftse_tidy_data['post_date'] = pd.to_datetime(ftse_tidy_data['post_date'])


ftse_tidy_data =\
    ftse_tidy_data.merge(
        ftse_rebal[['ISIN', 'Post Date', 'FTSE100', 'FTSE250']],
        how = 'left',
        left_on = ['ISIN', 'post_date'],
        right_on = ['ISIN', 'Post Date']
    )

ftse_tidy_data.head()

,Name,ISIN,post_date,pre_twenty_pd,pre_five_pd,pre_three_pd,pre_one_pd,post_three_pd,post_five_pd,post_ten_pd,post_twenty_pd,Post Date,FTSE100,FTSE250
0,ASOS,GB0030927254,2023-06-19,4.460999,3.292998,3.279998,3.703999,4.087000,3.927998,3.758999,3.518999,2023-06-19,NaN,-1.0
1,BRITISH LAND,GB0001367019,2023-06-19,3.562000,3.438999,3.347000,3.430000,3.083999,2.975000,3.132998,3.188999,2023-06-19,-1.0,1.0
2,CAPITA,GB00B23K0M20,2023-06-19,0.321000,0.329600,0.324400,0.320000,0.292000,0.279200,0.275000,0.278200,2023-06-19,NaN,1.0
3,CAPRICORN ENERGY,GB00BQ98V038,2023-06-19,1.950000,1.934000,1.920000,1.814000,1.818000,1.828000,1.878000,1.922000,2023-06-19,NaN,-1.0
4,EMPIRIC STUDENT PROPERTY,GB00BLWDVR75,2023-06-19,0.916000,0.884000,0.869000,0.919000,0.860000,0.850000,0.840000,0.875000,2023-06-19,NaN,1.0


## Identify stocks to remove due to corporate actions

In [15]:
# import corporate action data
corp_action = pd.read_csv('../output/corporate_actions_summary2.csv',index_col = 0)
corp_action.reset_index(drop = True, inplace=True)

for date in ['announceddate', 'exdate']:
    corp_action[date] = pd.to_datetime(corp_action[date])
    
corp_action.head()

,security_code,security_name,primary_exchange,refinitiv_code,isin_code,announceddate,exdate,corpactcode,corporateaction
0,48655.0,SYNTHOMER,LON,46186.0,GB00BNTVWJ75,2023-09-07,2023-09-26,CONS,Consolidation
1,48655.0,SYNTHOMER,LON,46186.0,GB00BNTVWJ75,2023-09-07,2023-09-28,RGHT,Rights Issue
2,48655.0,SYNTHOMER,LON,46186.0,GB00BNTVWJ75,2023-09-07,2023-09-26,WRDN,Write Down
3,6543.0,CAPRICORN ENERGY,LON,46290.0,GB00BQ98V038,2023-04-27,2023-05-16,REVS,Reverse Stock Split
4,6543.0,CAPRICORN ENERGY,LON,46290.0,GB00BQ98V038,2023-04-27,2023-05-16,CAPR,Capital Repayment


In [16]:
corp_action[corp_action['isin_code'] == isin].empty

False

In [17]:
indices_to_exclude = []

for idx, (isin, post_date) in enumerate(zip(ftse_tidy_data['ISIN'], ftse_tidy_data['post_date'])):
    
    if not corp_action[corp_action['isin_code'] == isin].empty:
        for announceddate, exdate in zip(corp_action['announceddate'], corp_action['exdate']):
            if announceddate <= post_date <= exdate:
                indices_to_exclude.append(idx)
                break
      

In [18]:
ftse_tidy_data = ftse_tidy_data.drop(indices_to_exclude)
ftse_tidy_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499 entries, 0 to 527
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Name            499 non-null    object        
 1   ISIN            499 non-null    object        
 2   post_date       499 non-null    datetime64[ns]
 3   pre_twenty_pd   499 non-null    float64       
 4   pre_five_pd     499 non-null    float64       
 5   pre_three_pd    499 non-null    float64       
 6   pre_one_pd      499 non-null    float64       
 7   post_three_pd   499 non-null    float64       
 8   post_five_pd    499 non-null    float64       
 9   post_ten_pd     499 non-null    float64       
 10  post_twenty_pd  499 non-null    float64       
 11  Post Date       499 non-null    datetime64[ns]
 12  FTSE100         137 non-null    float64       
 13  FTSE250         488 non-null    float64       
dtypes: datetime64[ns](2), float64(10), object(2)
memory usage:

In [19]:
(
        historical_prices
        .to_csv(
            '../output/historical_prices_ftse_rebal_10y.csv',
            index = False
            )
)



In [20]:
ftse_tidy_data.to_csv(
            '../output/ETL_ftse_price_data_10y.csv',
            index = False
            )